# Chapter 10. Data Aggregation and Group Operations
### split\-apply\-combine:<br>
1. **Data contained in a pandas object,whether a Series, DataFrame,or other wise is *split* into groups based on one or more *keys***
1. **A function is *applied* to each group, producing new value**
1. **The result of those function applications are combined into a result object**

**Each grouping key can take many forms, and the keys do not have to be all of the same type:**<br>
* **A value indicating a column name in a DataFrame**<br>
* **A dict or Series giving a correspondence between the values on the axis being grouped and the group names**<br>
* **A function to be invoked on the axis index or the individual labels in the index**<br>

In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.DataFrame({'key1':['a','b','b','b','a'],
               'key2':['one','two','one','two','one'],
               'data1':np.random.randn(5),
               'data2':np.random.randn(5)})

In [4]:
df

,key1,key2,data1,data2
0,a,one,1.718039,0.501036
1,b,two,1.254990,-0.008837
2,b,one,-0.361355,-0.424843
3,b,two,1.108816,-0.437780
4,a,one,-1.130581,-0.997790


**To compute the mean of the** data1 **column using the labels from** key1

In [5]:
grouped=df['data1'].groupby(df['key1'])

In [6]:
grouped

* **Returns a group by object**
* **Object has all the information needed to then apply some operation to each of the groups**<br>
* **Data Series has been aggregated according to the group key, producing a new Series that is now indexed by the unique values in the group by key  column**

In [7]:
grouped.mean()

key1
a    0.293729
b    0.667484
Name: data1, dtype: float64

**Passing multiple arrays as a list**
* **Resulting Series has a hierarchical index consisting of the unique pairs of keys observed**

In [8]:
means=df['data1'].groupby([df['key1'],df['key2']]).mean()

In [9]:
means

key1  key2
a     one     0.293729
b     one    -0.361355
      two     1.181903
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,one,two
key1,,
a,0.293729,NaN
b,-0.361355,1.181903


**The group keys can be any arrays of the right length**

In [13]:
states=np.array(['Ohio','California','California','Ohio','Ohio'])

In [14]:
years=np.array([2005,2005,2006,2005,2006])

In [15]:
states

array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'], dtype='<U10')

In [16]:
years

array([2005, 2005, 2006, 2005, 2006])

In [17]:
df['data1'].groupby([states,years]).mean()

California  2005    1.254990
            2006   -0.361355
Ohio        2005    1.413428
            2006   -1.130581
Name: data1, dtype: float64

* **The grouping information is found in the same DataFrame as the data you want to work on**
* **You can pass column names(whether those are strings,numbers, or other python objects) as the group keys**
* **All of the numeric columns are aggregated, though it is possible to filter down to a subset**
* **By default, all of the numeric columns are aggregated**

In [18]:
df.groupby('key1').mean()#'key2 ' is a nuisance column

,data1,data2
key1,,
a,0.293729,-0.248377
b,0.667484,-0.290486


In [19]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.293729 -0.248377
b    one  -0.361355 -0.424843
     two   1.181903 -0.223308

size **: returns a Series containing group sizes**

In [20]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
b     one     1
      two     2
dtype: int64

### Iterating Over Groups
**Groupy By object supports iteration, generating a sequence of 2-tuples containing the group name with the chunk of data**

In [21]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.718039  0.501036
4    a  one -1.130581 -0.997790
b
  key1 key2     data1     data2
1    b  two  1.254990 -0.008837
2    b  one -0.361355 -0.424843
3    b  two  1.108816 -0.437780


**In the case of multiple keysm the first element in the tuple will be a tuple of key values**

In [22]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print(k1,k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one  1.718039  0.501036
4    a  one -1.130581 -0.997790
b one
  key1 key2     data1     data2
2    b  one -0.361355 -0.424843
b two
  key1 key2     data1     data2
1    b  two  1.254990 -0.008837
3    b  two  1.108816 -0.437780


**Computing a dict of the data pieces as one-liner**

In [23]:
pieces=dict(list(df.groupby('key1')))

In [24]:
pieces['a']

,key1,key2,data1,data2
0,a,one,1.718039,0.501036
4,a,one,-1.130581,-0.997790


**By default,** groupby **groups on** axis=0 **,but you can group on any of the other axis**

In [25]:
groupped=df.groupby(df.dtypes,axis=1)

In [26]:
for dtype,group in groupped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.718039  0.501036
1  1.254990 -0.008837
2 -0.361355 -0.424843
3  1.108816 -0.437780
4 -1.130581 -0.997790
object
  key1 key2
0    a  one
1    b  two
2    b  one
3    b  two
4    a  one


### Selecting a column or a Subset of Columns

**Indexing a GroupBy object created from a DataFrame with a column name or array name has the effect of column subsetting for aggregation**

In [27]:
df.groupby('key1')['data1']#synthetic sugar for df['data1'].groupby('key1')

In [32]:
df.groupby(df['key1'])[['data2']]#synthetic sugar for df[['data2']].groupby(df['key1'])

**To compute the means for just the data2 column and get the result as a DataFrame**

In [35]:
df.groupby([df['key1'],df['key2']])[['data2']].mean()

data2
key1 key2          
a    one  -0.248377
b    one  -0.424843
     two  -0.223308

**Grouping with Dicts and Series**

In [36]:
people=pd.DataFrame(np.random.rand(5,5),
            columns=['a','b','c','d','e'],
            index=['Joe','Steven','Wes','Jim','Travis'])

In [37]:
people

,a,b,c,d,e
Joe,0.497571,0.484691,0.084113,0.114040,0.937601
Steven,0.706125,0.801951,0.106753,0.091092,0.744641
Wes,0.759035,0.042920,0.282309,0.075860,0.337049
Jim,0.993690,0.621283,0.610840,0.417877,0.802314
Travis,0.419763,0.799182,0.110093,0.473337,0.820097


In [38]:
people.iloc[2:3,[1,2]]=np.nan#Add a few NA values

In [39]:
people

,a,b,c,d,e
Joe,0.497571,0.484691,0.084113,0.114040,0.937601
Steven,0.706125,0.801951,0.106753,0.091092,0.744641
Wes,0.759035,NaN,NaN,0.075860,0.337049
Jim,0.993690,0.621283,0.610840,0.417877,0.802314
Travis,0.419763,0.799182,0.110093,0.473337,0.820097


**A group correspondence for the columns and sum together the columns by group**

In [40]:
mapping={'a':'red','b':'red','c':'blue',
         'd':'blue','e':'red','f':'orange'}

In [41]:
by_column=people.groupby(mapping,axis=1)

In [42]:
by_column.sum()

,blue,red
Joe,0.198153,1.919864
Steven,0.197845,2.252718
Wes,0.075860,1.096083
Jim,1.028717,2.417286
Travis,0.583430,2.039041


**Series as group keys**

In [43]:
map_Series=pd.Series(mapping)

In [44]:
map_Series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [46]:
people.groupby(map_Series,axis=1).count()

,blue,red
Joe,2,3
Steven,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions
**Any function passed as a group key will be called once per index value, with the return value being used as the group names**

**Example: group by the length of the names**

In [47]:
people.groupby(len).sum()

,a,b,c,d,e
3,2.250296,1.105974,0.694953,0.607778,2.076964
6,1.125888,1.601133,0.216846,0.564430,1.564738


**Mixing functions with arrays, dicts, or Series is not a problem as everthing gets converted to arrays internally**

In [48]:
key_list=['one','one','one','two','two']

In [49]:
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one  0.497571  0.484691  0.084113  0.075860  0.337049
  two  0.993690  0.621283  0.610840  0.417877  0.802314
6 one  0.706125  0.801951  0.106753  0.091092  0.744641
  two  0.419763  0.799182  0.110093  0.473337  0.820097

### Grouping by Index Levels

**Aggregate using one of the levels of an axis index**

In [51]:
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                          [1,3,5,1,3]],
                          names=['city','tenor'])

In [52]:
hier_df=pd.DataFrame(np.random.randn(4,5),columns=columns)

In [53]:
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      1.374911 -1.391648  0.413488 -0.056548 -0.970559
1      0.958934 -0.542359 -0.197148  1.431276 -0.021090
2     -0.826356  1.427909 -0.224244  0.131109  0.204505
3     -0.135131  0.333169  0.564675 -1.085332  0.577270

**To group by level, pass the level number or name using the** level **keyword**

In [54]:
hier_df.groupby(level='city',axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation
* **Aggregation refer to any data transformation that produces scalar values from arrays**
* **You can use aggregation of your devising and additionally call any method that is also defined on the grouped object**

#### Use Quantile

* quantile **is not explicitly implemented for GroupBy, it is a Series method and thus availiable for use**
* **GroupyBy efficiently slices up the Series, calls** pieces.quantile(0.9) **for each piece, and then assembles those results rogether into the result object**

In [55]:
df

,key1,key2,data1,data2
0,a,one,1.718039,0.501036
1,b,two,1.254990,-0.008837
2,b,one,-0.361355,-0.424843
3,b,two,1.108816,-0.437780
4,a,one,-1.130581,-0.997790


In [56]:
grouped=df.groupby('key1')

**Implement you aggregation function: Pass any function that aggregates an array to the** aggregate **or** agg **method**

In [57]:
def peak_to_peak(arr):
    return arr.max()-arr.min()

In [58]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.848620,1.498825
b,1.616345,0.428943


In [59]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      2.0  0.293729  2.014279 -1.130581 -0.418426  0.293729  1.005884   
b      3.0  0.667484  0.893993 -0.361355  0.373730  1.108816  1.181903   

               data2                                                   \
           max count      mean       std      min       25%       50%   
key1                                                                    
a     1.718039   2.0 -0.248377  1.059830 -0.99779 -0.623083 -0.248377   
b     1.254990   3.0 -0.290486  0.244002 -0.43778 -0.431311 -0.424843   

                          
           75%       max  
key1                      
a     0.126329  0.501036  
b    -0.216840 -0.008837

### Column-Wise and Multiple Function Application

In [61]:
tips=pd.read_csv('examples/tips.csv')

In [62]:
#Add tip percentage of total bill
tips['tip_pct']=tips['tip']/tips['total_bill']

In [63]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [64]:
grouped=tips.groupby(['day','smoker'])

**For already established descriptive statistics, you can pass the name of the function as a string**

In [66]:
group_pct=grouped['tip_pct']

In [67]:
group_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

**If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions**

In [69]:
group_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

**If you pass a list of (name,function) tuples,the first element of each tuple will be used as the DataFrame column name**

In [70]:
group_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

**With a DataFrame, you can specify a list of functions to apply to all of the columns or different functions per columns**

In [71]:
functions=['count','mean','max']

In [76]:
result=grouped[['tip_pct','total_bill']].agg(functions)

In [77]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

**The resulting DataFrame has a hierarchical columns, the same as you would get aggregating each column separately using** concat **to glue the results using the column names as the** keys **argument**

In [78]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

**A list of tuples with custom names can be passed**

In [79]:
ftuples=[('Durchschnitt','mean'),('Abweichung',np.var)]

In [80]:
grouped['tip_pct','total_bill'].agg(ftuples)

<ipython-input-80-66716e8bd20a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct','total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

**Apply potentially different functions to one or more of the columns**<br>
**Pass a dict to agg that contains a mapping of column names to any of the function specification listed so far**

In [81]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

**A DataFrame will have hierarchical columns only if multiple functions are applied to at least one column**

In [82]:
grouped.agg({'tip_pct':['min','max','mean','std'],'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes
**Disable the index by passing** as_index=False **to** groupby

In [84]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply: General split-apply-combine
apply **splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces together**

**Example: Select the top five** tip_pct **values by group**

In [86]:
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top **function is called on each row group from the DataFrame, and then the results are glued together using** pandas.concat, **labeling the pieces with the group names**

In [87]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

**If you pass a function that takes other arguments or keywotds, you can pass these after the function**

In [88]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

**Inside GroupBy, when you invoke method like** describe, **it is actually just a shortcut for:**

In [89]:
f=lambda x:x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

In [91]:
grouped.describe().unstack()

total_bill                                                          \
            count             mean                  std               min   
smoker         No   Yes         No        Yes        No        Yes     No   
day                                                                         
Fri           4.0  15.0  18.420000  16.813333  5.059282   9.086388  12.46   
Sat          45.0  42.0  19.661778  21.276667  8.939181  10.069138   7.25   
Sun          57.0  19.0  20.506667  24.120000  8.130189  10.442511   8.77   
Thur         45.0  17.0  17.113111  19.190588  7.721728   8.355149   7.51   

                              ...   tip_pct                                \
                 25%          ...       min                 25%             
smoker    Yes     No     Yes  ...        No       Yes        No       Yes   
day                           ...                                           
Fri      5.75  15.10  11.690  ...  0.120385  0.103555  0.137239  0.133739   
Sat      3.07  14.73  13.405  ...  0.056797  0.035638  0.136240  0.091797   
Sun      7.25  14.78  17.165  ...  0.059447  0.065660  0.139780  0.097723   
Thur    10.34  11.69  13.510  ...  0.072961  0.090014  0.137741  0.148038   

                                                                    
             50%                 75%                 max            
smoker        No       Yes        No       Yes        No       Yes  
day                                                                 
Fri     0.149241  0.173913  0.163652  0.209240  0.187735  0.263480  
Sat     0.150152  0.153624  0.183915  0.190502  0.291990  0.325733  
Sun     0.161665  0.138122  0.185185  0.215325  0.252672  0.710345  
Thur    0.153492  0.153846  0.184843  0.194837  0.266312  0.241255  

[4 rows x 64 columns]

### Suppresing the Group Keys
**Passing** group_keys=False **to** groupby

In [92]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [93]:
frame=pd.DataFrame({'data1':np.random.randn(1000),
                   'data2':np.random.randn(1000)})

In [100]:
quartiles=pd.cut(frame.data1,4)

In [101]:
quartiles[:10]

0    (-1.599, -0.0286]
1       (1.542, 3.112]
2    (-1.599, -0.0286]
3    (-1.599, -0.0286]
4    (-1.599, -0.0286]
5     (-3.175, -1.599]
6     (-0.0286, 1.542]
7    (-1.599, -0.0286]
8     (-0.0286, 1.542]
9     (-3.175, -1.599]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.175, -1.599] < (-1.599, -0.0286] < (-0.0286, 1.542] < (1.542, 3.112]]

**The** Categorical **object retruned by** cut **can be passed directly to groupby**

In [99]:
def get_stats(group):
    return {'min':group.min(),'max':group.max(),
           'count':group.count(),'mean':group.mean()}

In [102]:
grouped=frame.data2.groupby(quartiles)

In [103]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.175, -1.599]",-1.940894,2.299854,46.0,0.113039
"(-1.599, -0.0286]",-2.850250,2.672097,445.0,0.027930
"(-0.0286, 1.542]",-2.718521,3.029440,447.0,-0.037452
"(1.542, 3.112]",-2.848828,1.835657,62.0,-0.075396


### Example: Filling Missing Values with Group-Specific Values

**Impute the null(NA) values using a fixed value or some value derived from the data**

In [104]:
s=pd.Series(np.random.randn(6))

In [105]:
s[::2]=np.nan

In [106]:
s

0         NaN
1    2.002984
2         NaN
3    0.499913
4         NaN
5   -1.188772
dtype: float64

In [107]:
s.fillna(s.mean())

0    0.438042
1    2.002984
2    0.438042
3    0.499913
4    0.438042
5   -1.188772
dtype: float64

**Suppose you need to fill value to vary by group. One way is to group the data and use** apply **with a function that calls** fillna **on each chunck**

In [108]:
states=['Ohio','New York', 'Vermont','Florida',
       'Oregon','Nevada','California','Idaho']

In [109]:
group_key=['East']*4+['West']*4

In [110]:
data=pd.Series(np.random.randn(8),index=states)

In [111]:
data

Ohio         -0.650865
New York      1.659409
Vermont      -0.695168
Florida       1.300876
Oregon       -0.127990
Nevada       -0.818631
California   -0.122646
Idaho         2.598562
dtype: float64

In [112]:
data['Vermont','Nevada','Idaho']=np.nan

In [113]:
data

Ohio         -0.650865
New York      1.659409
Vermont            NaN
Florida       1.300876
Oregon       -0.127990
Nevada             NaN
California   -0.122646
Idaho              NaN
dtype: float64

In [114]:
data.groupby(group_key).mean()

East    0.769807
West   -0.125318
dtype: float64

**Fill the NA values using the group means**

In [115]:
fill_mean=lambda g:g.fillna(g.mean())

In [116]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.650865
New York      1.659409
Vermont       0.769807
Florida       1.300876
Oregon       -0.127990
Nevada       -0.125318
California   -0.122646
Idaho        -0.125318
dtype: float64

**Filling in values that vary by group**

In [117]:
fill_values={'East':0.5,'West':-1}

In [118]:
fill_func=lambda g:g.fillna(fill_values[g.name])

In [119]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.650865
New York      1.659409
Vermont       0.500000
Florida       1.300876
Oregon       -0.127990
Nevada       -1.000000
California   -0.122646
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

**Draw a random sample(with or without replacement) from a large dataset**

In [122]:
#Hearts, Spades, Clubs, Diamonds
suits=['H','S','C','D']
card_val=(list(range(1,11))+[10]*3)*4
base_names=['A']+list(range(2,11))+['J','K','Q']
cards=[]
for suit in ['H','S','C','D']:
    cards.extend(str(num)+suit for num in base_names)

deck=pd.Series(card_val,index=cards)

**Returns a Series of length 52 whose index contains card names and values are the ones used in Blackjack and other games**

In [123]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

**Drawing a hand of five cards from the deck**

In [124]:
def draw(deck,n=5):
    return deck.sample(n)

In [125]:
draw(deck)

8H     8
AC     1
QD    10
JS    10
QS    10
dtype: int64

**Draw random cards from each suit**

In [126]:
get_suit=lambda card: card[-1]#last letter is suit

In [127]:
deck.groupby(get_suit).apply(draw,n=2)

C  10C    10
   JC     10
D  4D      4
   AD      1
H  8H      8
   2H      2
S  4S      4
   2S      2
dtype: int64

**Or we could write**

In [128]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

AC      1
7C      7
KD     10
2D      2
4H      4
9H      9
AS      1
10S    10
dtype: int64

### Example: Group Weighted Average and Correlation
**Under** groupby **operations between columns in a DataFram or two Series, such as a group weighter average are possible**

In [129]:
df=pd.DataFrame({'category':['a','a','a','a',
                            'b','b','b','b'],
                'data':np.random.randn(8),
                'weights':np.random.randn(8)})

In [130]:
df

,category,data,weights
0,a,-1.051795,0.779509
1,a,0.529583,-0.164845
2,a,0.990108,1.269197
3,a,-0.243200,0.764881
4,b,1.731356,1.301740
5,b,1.832711,-1.083975
6,b,-0.611554,0.764761
7,b,0.132299,1.413590


In [131]:
grouped=df.groupby('category')

**Compute the weighted average**

In [132]:
get_wavg=lambda g:np.average(g['data'],weights=g['weights'])

In [133]:
grouped.apply(get_wavg)

category
a    0.061705
b   -0.005640
dtype: float64

#### Compute Correlation

In [135]:
close_px=pd.read_csv('examples/stock_px_2.csv',parse_dates=True,
                    index_col=0)

In [136]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [137]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


**Compute a DataFram consisting of the yearly correlations of daily returns(computed from percent changes) with SPX**<br>
**1. Creaye a function that computes the pairwise correlation of each column with** 'SPX' **column**

In [138]:
spx_corr=lambda x:x.corrwith(x['SPX'])

**Compute percent change on** close_px **using** pct_change

In [140]:
rets=close_px.pct_change().dropna()

**Group the percent changes by year, which can be extracted from each row lable with one-line function that returns the year attribute of each** datetime **label**

In [141]:
get_year=lambda x:x.year

In [142]:
by_year=rets.groupby(get_year)

In [143]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


**Compute the inter-column correlations**

In [145]:
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [148]:
import statsmodels.api as sm

In [153]:
def regress(data,yvar,xvars):
    Y=data[yvar]
    X=data[xvars]
    X['intercept']=1
    result=sm.OLS(Y,X).fit()
    return result.params

**Yearly linear regression of** AAPL **on** SPX **returns**

In [154]:
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


### 10.4 Pivot Tables and Cross-Tabulation

***Pivot table*: Aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns**

**Compute a table of group means(the default** pivot_table **aggregation type) arranged by** day **and** smoker **on rows**

In [155]:
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [189]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

**We could augment this table to include partial totals by passing** margins=True <br>
* **This has the effect of adding** All **row and column lables, with the corresponding values being the group statistics for all the data within the single tier**
* **The** All **values are means without taking into account smoker versus non-smoker(the** All **columns) or any two levels of the grouping on the rows(the** All **row)**

In [190]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

**To use a different aggregate function, pass it to** aggfunc 

In [191]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',
                aggfunc=len,margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

**If some combinations are empty, you may wish to pass fill_value:**

In [192]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],
                columns='day',aggfunc='mean',fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab
**A cross-tabulation(or crosstab for short) is a special case of pivot table that computes group frequencies**

In [196]:
data=pd.DataFrame({'Sample':[1,2,3,4,5,6,7,8,9,10],
                  'Nationality':['USA','Japan','USA','Japan','Japan','Japan','USA','USA','Japan','USA'],
                  'Handedness':['Right-handed','Left-handed','Right-handed','Right-handed','Left-handed',
                  'Right-handed','Right-handed','Left-handed','Right-handed','Right-handed']})

In [197]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


**Summarize this data by nationality and handedness**

In [198]:
pd.crosstab(data.Nationality,data.Handedness,margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


**The first two arguments to** crosstab **can each either be an array or Series or a list of arrays**

In [199]:
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [200]:
%%bash
git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    .ipynb_checkpoints/Untitled-checkpoint.ipynb
	modified:   Data_Loading_Practice.ipynb
	modified:   NumPy_Basics_Arrays_and_Vectorized_Computation.ipynb
	deleted:    Untitled.ipynb
	deleted:    Untitled1.ipynb
	modified:   data_cleaning.ipynb
	modified:   datawrangling.ipynb
	modified:   plotting_and_visulization.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.DS_Store
	.ipynb_checkpoints/Data_Loading_Practice-checkpoint.ipynb
	.ipynb_checkpoints/NumPy_Basics_Arrays_and_Vectorized_Computation-checkpoint.ipynb
	.ipynb_checkpoints/TimeSeries (1column)-checkpoint.ipynb
	.ipynb_checkpoints/data_cleaning-checkpoint.ipynb
	.ipynb_checkpoints/datawrangling-checkpoint.ipynb
	.ipynb_checkpoints/group_operation-c

In [ ]:
%%bash
git add "t"